In [8]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

In [13]:
workdir = '/petrobr/parceirosbr/bigoilict/users/cristian.villalobos/personality-prediction-from-text'
results_r1 = os.path.join(workdir, 'data_output_dataset_regular_answers_1')
results_r2 = os.path.join(workdir, 'data_output_dataset_regular_answers_2')
results_r3 = os.path.join(workdir, 'data_output_dataset_regular_answers_3')
results_dirs = [results_r1, results_r2, results_r3]

In [14]:
for r_dir in [results_r1,results_r2,results_r3]:
     for i in range(10):
        x_path = os.path.join(r_dir,f'prediction_{i}.xlsx')
        if not os.path.exists(x_path):
            print(x_path)

/petrobr/parceirosbr/bigoilict/users/cristian.villalobos/personality-prediction-from-text/data_output_dataset_regular_answers_1/prediction_2.xlsx
/petrobr/parceirosbr/bigoilict/users/cristian.villalobos/personality-prediction-from-text/data_output_dataset_regular_answers_2/prediction_1.xlsx


In [15]:
output_folder='output_dataset_regular_answers'
os.makedirs(output_folder, exist_ok=True)

In [25]:
from tqdm import tqdm
all_df={}
for r_dir in tqdm(results_dirs):
    
    df = []
    for i in range(10):
        path = os.path.join(r_dir,f'prediction_{i}.xlsx')
        if os.path.exists(path):
            df.append(pd.read_excel(path))
    
    df = pd.concat(df) 
    df['pred_sES'] = df['pred_sNEU'].apply(lambda x:5-x)
    df['pred_cES'] = df['pred_cNEU']
    all_df[r_dir]=df

100%|██████████| 3/3 [02:04<00:00, 41.40s/it]


# Classification vs Starting words groups

In [36]:
import matplotlib.pyplot as plt
import numpy as np

traits = ['OPN', 'CON', 'EXT', 'AGR', 'NEU']
traits_ok = ['OPN', 'CON', 'EXT', 'AGR', 'ES']
categories = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional stability']
regular_categories = ["Tell me about yourself", "Cultural fit", "Strengths and weaknesses/challenges", \
 "Plans for the future (where do you see yourself in X years)", "Handing pressure "]
model_names = ['gpt2','gpt2_medium','gpt2_large','op_125m','op_350m','op_13b', 'xlnet', 'xlnet_large']

ind = np.arange(len(traits_ok))
for r_index,r_dir in enumerate(results_dirs):
    
    df = all_df[r_dir]
    
    for k,g in enumerate(model_names):

        gdf=df[df['model_name']==g]

        fig, axs = plt.subplots(5,1, sharex=False, sharey=False, figsize=(15,20), frameon=False)
        fig.subplots_adjust(left=0.25)
        
        for i,rcat in enumerate(regular_categories):
            cdf = gdf[gdf['category']==rcat]
            scores = []
            for j,trait in enumerate(traits_ok):
                x = cdf['pred_c' + trait]
                if len(x)==0:
                    x=0
                else:
                    x = x.sum()/len(x)
                scores.append(x)
            print(scores)
            axs[i].bar(ind, scores)
                     
            axs[i].set_title(cat)

            #axs[i].set_xlim([1,5])
            axs[i].set_xticks(ind)
            axs[i].set_xticklabels(traits_ok)
            axs[i].set_ylim([0,1])

        plt.savefig(os.path.join(output_folder, f'sents_{r_index+1}_model_{g}_clf_vs_groups.jpg'))

[0.9714, 0.3354, 0.2516, 0.4398, 0.0932]
[0.9746, 0.3176, 0.1566, 0.7306, 0.0458]
[0.9658, 0.25, 0.1304, 0.6448, 0.0866]
[0.9742, 0.2412, 0.1312, 0.6266, 0.1272]
[0.9682, 0.3356, 0.2906, 0.5444, 0.1572]
[0.977, 0.3294, 0.2352, 0.4496, 0.0976]
[0.9682, 0.2974, 0.1514, 0.7264, 0.0546]
[0.9682, 0.2284, 0.1296, 0.6436, 0.1074]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0.9706, 0.2322, 0.1322, 0.6434, 0.086]
[0.9728, 0.2598, 0.1158, 0.6528, 0.1106]
[0.969, 0.3258, 0.3028, 0.5458, 0.1372]
[0.962, 0.209, 0.2618, 0.3788, 0.1806]
[0.9706, 0.1274, 0.2058, 0.8458, 0.046]
[0.9538, 0.1002, 0.0856, 0.4948, 0.1068]
[0.9626, 0.1186, 0.073, 0.5334, 0.1362]
[0.9234, 0.3458, 0.3608, 0.3506, 0.3214]
[0.9774, 0.2824, 0.2848, 0.4262, 0.1102]
[0.9712, 0.2616, 0.1802, 0.7248, 0.0564]
[0.9698, 0.2164, 0.1336, 0.632, 0.1036]
[0.9752, 0.215, 0.1192, 0.612, 0.1236]
[0.9638, 0.3066, 0.3226, 0.56, 0.1614]
[0.9698, 0.2688, 0.2034, 0.449, 0.1688]
[0.9914, 0.2384, 0.1402, 0.8042, 0.063]
[0.9846, 

# Average Personality for each Model

In [58]:
traits = ['OPN', 'CON', 'EXT', 'AGR', 'NEU']
traits_ok = ['OPN', 'CON', 'EXT', 'AGR', 'ES']
categories = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional stability']
regular_categories = ["Tell me about yourself", "Cultural fit", "Strengths and weaknesses/challenges", \
 "Plans for the future (where do you see yourself in X years)", "Handing pressure "]
model_names = ['gpt2','gpt2_medium','gpt2_large','op_125m','op_350m','op_13b', 'xlnet', 'xlnet_large']
results_dirs = [results_t1, results_t2, results_t3]

all_xvalues = {}

for r_index,r_dir in enumerate(results_dirs):
    
    all_xvalues[r_index+1] = {}
    
    df = []
    for i in range(10):
        df.append(pd.read_excel(os.path.join(r_dir,f'prediction_{i}.xlsx')))
    df = pd.concat(df) 
    df['pred_sES'] = df['pred_sNEU'].apply(lambda x:5-x)
    for k,g in enumerate(model_names):
        
        gdf=df[df['model_name']==g]
        
        all_xvalues[r_index+1][k] = {} 
        
        for cat in regular_categories:
            
            xmvalues = []
            xvvalues = []
            for trait in traits_ok:
            
                x = gdf[gdf['category']==cat]['pred_s'+trait].values
                
                xmvalues.append(np.mean(x))
                xvvalues.append(np.std(x))
            
            all_xvalues[r_index+1][k][cat] = {'mean':xmvalues,'std':xvvalues}

In [46]:
names = list(map(lambda x:x[0],traits_ok))

In [71]:
fig, axs = plt.subplots(len(model_names),1, sharex=False, sharey=False, figsize=(15,30), frameon=False)
#fig.subplots_adjust(left=0.25)   
colors = ['g','b','k']
width = 0.27
for k,model in enumerate(model_names):
    ind = np.arange(5) 
    rects = []
    for n in range(3):
        vals = np.array(all_xvalues[n+1][k]['mean'])
        err = np.array(all_xvalues[n+1][k]['std'])
        rect = axs[k].bar(ind+width*n, vals, width, yerr=err, color=colors[n])
        rects.append(rect)
    
    axs[k].set_title(model)
    
    axs[k].set_ylabel('Scores')
    axs[k].set_xticks(ind+width)
    axs[k].set_xticklabels( names )
    axs[k].legend( (rects[0][0], rects[1][0], rects[2][0]), ('1 sent', '2 sents', '3 sents') )
plt.savefig(f'personality_by_nsents.jpg')

# Higher and Lower Scores Examples

In [79]:
traits = ['OPN', 'CON', 'EXT', 'AGR', 'NEU']
traits_ok = ['OPN', 'CON', 'EXT', 'AGR', 'ES']
categories = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional stability']
model_names = ['gpt2','gpt2_medium','gpt2_large','op_125m','op_350m','op_13b', 'xlnet', 'xlnet_large']
results_dirs = [results_t1, results_t2, results_t3]
all_examples = {}

for r_index,r_dir in enumerate(results_dirs):
    
    all_examples[r_index+1] = {}
    
    df = []
    for i in range(10):
        df.append(pd.read_excel(os.path.join(r_dir,f'prediction_{i}.xlsx')))
    df = pd.concat(df) 
    df['pred_sES'] = df['pred_sNEU'].apply(lambda x:5-x)
    
    for k,g in enumerate(model_names):
        
        gdf=df[df['model_name']==g]
        
        examples = {}
        for i,(cat,trait) in enumerate(zip(categories, traits_ok)):

            xdf = gdf[gdf['category']==cat].sort_values(by='pred_s'+trait, ascending=False)
            
            higher = xdf.iloc[:3][['text','pred_s'+trait]]
            lower  = xdf.iloc[-3:][['text','pred_s'+trait]]
            
            examples[trait] = {'higher':higher, 'lower':lower}
            
        all_examples[r_index+1][k] = examples

In [101]:
for nsents in range(1,4):
    for case_name in ['higher','lower']:
        for trait in traits_ok:
            table = []
            for k,m in enumerate(model_names):
                case = all_examples[nsents][k][trait]['higher']
                for _,row in case.iterrows():
                    trow = {'text':row['pred_s'+trait] , 'score':row['text'], 'model':m}
                    table.append(trow)
            table = pd.DataFrame(table)
            table.to_excel(f"{nsents}_sents_{trait}_{case_name}.xlsx")

ES


,text,score,model
0,3.123513,"When I encounter a stressful situation, I will...",gpt2
1,3.026200,"When I am craving something, I usually feel I ...",gpt2
2,3.003500,Being the center of attention makes me feel so...,gpt2
3,3.126000,Being the center of attention makes me feel ba...,gpt2_medium
4,3.066144,My opinion of myself is what I have to offer ....,gpt2_medium
5,3.053399,"When I encounter a stressful situation, I can ...",gpt2_medium
6,3.156641,"When I am craving something, I usually have to...",gpt2_large
7,3.137559,"When I encounter a stressful situation, I need...",gpt2_large
8,3.065240,"When I encounter a stressful situation, I'm of...",gpt2_large
9,3.127756,"When I encounter a stressful situation, I smil...",op_125m


# Histograms

In [ ]:
import matplotlib.pyplot as plt
traits = ['OPN', 'CON', 'EXT', 'AGR', 'NEU']
traits_ok = ['OPN', 'CON', 'EXT', 'AGR', 'ES']
categories = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Emotional stability']

for r_index,r_dir in enumerate(results_dirs):
        
    df = []
    for i in range(10):
        df.append(pd.read_excel(os.path.join(r_dir,f'prediction_{i}.xlsx')))
    df = pd.concat(df) 
    
    df['pred_sES'] = df['pred_sNEU'].apply(lambda x:5-x)
    
    for k,g in enumerate(model_names):

        gdf=df[df['model_name']==g]
        
        plt.figure(figsize=(10,20))

        for i,(cat,trait) in enumerate(zip(categories, traits_ok)):
            print(cat,trait)
            plt.subplot(len(categories),1,i+1)
            x = gdf[gdf['category']==cat]['pred_s'+trait].values
            plt.hist(x)
            plt.title(cat)
            plt.xlim([1,5])
            
        plt.savefig(f'{r_index+1}_sents_{g}.jpg')